In [8]:
import numpy as np
import signac
import hoomd
import gsd.hoomd
import random
from cmeutils.dynamics import msd_from_gsd

In [2]:
project = signac.get_project("/home/stephaniemccallu/scratch/Entanglements")

In [30]:
def bootstrap_msd(job, start=0, stop=None, stride=1):
    steps_per_frame = int(5e5)
    ts = job.doc.real_time_step * 1e-15
    ts_frame = steps_per_frame * ts
    rand_list = []
    msd_list = []
    
    n = 10
    for i in range(n):
        rand_list.append(random.randint(0,199))
    for ind in rand_list:
        msd = msd_from_gsd(
            gsdfile=job.fn("production3-center.gsd"),
            start=ind,
            stop=(ind+200),
            atom_types="B",
            msd_mode="direct")
        msd_results = np.copy(msd.msd)
        np.array(msd_list)
        msd_list.append(msd_results)
    msd_avg = np.zeros((len(msd_list[0])))
    for j in msd_list:
        msd_avg += j
    msd_avg = msd_avg/len(msd_list)
    conv_factor = job.doc.ref_length**2
    job.doc.msd_units = "nm**2"
    msd_avg *= conv_factor
    time_array = np.arange(0, len(msd.msd), 1) * ts_frame
    np.save(file=job.fn("msd_time_bs.npy"), arr=time_array)
    np.save(file=job.fn("msd_data_real_nm_squared_bs.npy"), arr=msd_avg)
    np.save(file=job.fn("msd_data_reduced_bs.npy"), arr=msd_list)

In [31]:
for job in project.find_jobs():
    if job.isfile("production3-center.gsd"):
        bootstrap_msd(job)